In [ ]:
###### two robot imports

import pydot
from IPython.display import SVG, display
import pydrake.all
from pydrake.all import (DiagramBuilder, MeshcatVisualizer, Simulator,
                         StartMeshcat)

from manipulation.scenarios import MakeManipulationStation, AddIiwa, AddRgbdSensors, AddRgbdSensor, AddWsg

###### art imports

import numpy as np
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph, Box,
                         Capsule, ContactResults, DiagramBuilder,
                         InverseDynamicsController, JointSliders, LeafSystem,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         MultibodyPlant, Rgba, RigidTransform, RollPitchYaw,
                         RotationMatrix, Simulator, Sphere, StartMeshcat,
                         StateInterpolatorWithDiscreteDerivative, Parser, LoadModelDirectivesFromString)

from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingXyzJoint, AddShape


from pydrake.all import (
    AbstractValue, Adder, AddMultibodyPlantSceneGraph, BallRpyJoint, BaseField,
    Box, CameraInfo, ClippingRange, CoulombFriction, Cylinder, Demultiplexer,
    DepthImageToPointCloud, DepthRange, DepthRenderCamera, DiagramBuilder,
    FindResourceOrThrow, GeometryInstance, InverseDynamicsController,
    LeafSystem, LoadModelDirectivesFromString,
    MakeMultibodyStateToWsgStateSystem, MakePhongIllustrationProperties,
    MakeRenderEngineVtk, ModelInstanceIndex, MultibodyPlant, Parser,
    PassThrough, PrismaticJoint, ProcessModelDirectives, RenderCameraCore,
    RenderEngineVtkParams, RevoluteJoint, Rgba, RgbdSensor, RigidTransform,
    RollPitchYaw, RotationMatrix, SchunkWsgPositionController, SpatialInertia,
    Sphere, StateInterpolatorWithDiscreteDerivative, UnitInertia)
from pydrake.manipulation.planner import (
    DifferentialInverseKinematicsIntegrator,
    DifferentialInverseKinematicsParameters)

from manipulation.utils import AddPackagePaths, FindResource

from pydrake.all import (DiagramBuilder, Integrator, JacobianWrtVariable,
                         LeafSystem, MeshcatVisualizer, Simulator,
                         StartMeshcat)

from manipulation.meshcat_utils import AddMeshcatTriad

from pydrake.all import (AbstractValue, ConstantVectorSource, DiagramBuilder,
                         LeafSystem, MeshcatVisualizer,
                         MeshcatVisualizerParams, PiecewisePose,
                         RigidTransform, RotationMatrix, Simulator,
                         StartMeshcat, TrajectorySource)

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
model_directives = """
directives:
- add_model:
    name: iiwa0
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa0::iiwa_link_0
    X_PC:
        translation: [-1, 0, 0]
        rotation: !Rpy { deg: [0, 0, 90]}
- add_model:
    name: wsg0
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa0::iiwa_link_7
    child: wsg0::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
- add_model:
    name: iiwa1
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa1::iiwa_link_0
    X_PC:
        translation: [1, 0, 0]
        rotation: !Rpy { deg: [0, 0, -90]}
- add_model:
    name: wsg1
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa1::iiwa_link_7
    child: wsg1::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

"""

In [ ]:
# Writing System.
class MeshcatWriter(LeafSystem):

    def __init__(self, meshcat, drawing_body_index, canvas_body_index,
                    rgba, line_width=0.01, draw_threshold=0.01):
        LeafSystem.__init__(self)
        self._meshcat = meshcat
        self._drawing_body_index = drawing_body_index
        self._canvas_body_index = canvas_body_index
        self._rgba = rgba
        self._line_width = line_width
        self._draw_threshold = draw_threshold

        self.DeclareAbstractInputPort("contact_results",
                                        AbstractValue.Make(ContactResults()))
        self.DeclarePeriodicDiscreteUpdateEvent(0.01, 0, self.MaybeDraw)

        self._p_WLastDraw_index = self.DeclareDiscreteState(3)
        self._was_in_contact_index = self.DeclareDiscreteState([0])
        self._num_drawn_index = self.DeclareDiscreteState([0])

    def MaybeDraw(self, context, discrete_state):
        results = self.get_input_port().Eval(context)

        # The point on the canvas that most deeply penetrates the chalk.
        p_WDraw = None
        for i in range(results.num_point_pair_contacts()):
            info = results.point_pair_contact_info(i)
            if (info.bodyA_index() == self._drawing_body_index and
                    info.bodyB_index() == self._canvas_body_index):
                p_WDraw = info.point_pair().p_WCb
                break
            elif (info.bodyA_index() == self._drawing_body_index and
                    info.bodyB_index() == self._canvas_body_index):
                p_WDraw = info.point_pair().p_WCa
                break

        if p_WDraw is not None:
            p_WLastDraw = context.get_discrete_state(
                self._p_WLastDraw_index).get_value()
            was_in_contact = context.get_discrete_state(
                self._was_in_contact_index)[0]
            num_drawn = context.get_discrete_state(self._num_drawn_index)[0]

            length = np.linalg.norm(p_WDraw - p_WLastDraw)
            if (was_in_contact and length > self._draw_threshold):
                meshcat.SetObject(f"writer/{num_drawn}",
                                    Capsule(self._line_width, length),
                                    self._rgba)
                p_WMidpoint = (p_WDraw + p_WLastDraw)/2
                X_WCapsule = RigidTransform(
                    RotationMatrix.MakeFromOneVector(
                        p_WDraw - p_WLastDraw, 2), p_WMidpoint)
                meshcat.SetTransform(f"writer/{num_drawn}", X_WCapsule)
                discrete_state.set_value(self._num_drawn_index,
                                            [num_drawn + 1])
                discrete_state.set_value(self._p_WLastDraw_index, p_WDraw)
            elif not was_in_contact:
                discrete_state.set_value(self._p_WLastDraw_index, p_WDraw)

            discrete_state.set_value(self._was_in_contact_index, [1])
        else:
            discrete_state.set_value(self._was_in_contact_index, [0])

def ChalkController(time_step = 0.002):
    controller_plant = MultibodyPlant(time_step)
    AddShape(controller_plant, Capsule(0.01, 0.2), "chalk", mass=1, mu=1,
        color=[1, .34, .2, 1.0])
    AddFloatingXyzJoint(controller_plant,
                controller_plant.GetFrameByName("chalk"),
                controller_plant.GetModelInstanceByName("chalk"),
                actuators=True)

    controller_plant.Finalize()

    return controller_plant

In [ ]:
def AddIiwas(plant):
    parser = Parser(plant)
    AddPackagePaths(parser)
    if model_directives:
        directives = LoadModelDirectivesFromString(model_directives)
        ProcessModelDirectives(directives, parser)

def AddEnvironment(plant):
    # add table
    AddShape(plant, Box(1.5, 1.5, .2), "table", color=[0.05, 0.05, 0.05, 1.0])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("table"),
                        RigidTransform([0, 0, .1]))

    # add chalk
    chalk_instance = AddShape(plant, Capsule(0.01, 0.2), "chalk", mass=1, mu=1,
            color=[1.0, 0.34, 0.2, 1.0])
    gripper0 = plant.GetModelInstanceByName("wsg0")
    plant.WeldFrames(plant.GetFrameByName("body", gripper0), plant.GetFrameByName("chalk"),
                        RigidTransform([0, 0.075, 0]))

    # add eraser
    eraser_instance = AddShape(plant, Box(0.05, 0.025, 0.125), "eraser", mass=1, mu=1,
            color=[1.0, 1.0, 1.0, 1.0])
    gripper1 = plant.GetModelInstanceByName("wsg1")
    plant.WeldFrames(plant.GetFrameByName("body", gripper1), plant.GetFrameByName("eraser"),
                        RigidTransform([0, 0.125, 0]))
    #AddFloatingXyzJoint(plant,
    #                    plant.GetFrameByName("chalk"),
    #                    chalk_instance,
    #                    actuators=True)
    return chalk_instance 

def IiwaControllerSetup(builder, plant, scene_graph, time_step = 0.002):
    iiwa_prefix = "iiwa"
    wsg_prefix = "wsg"

    for i in range(plant.num_model_instances()):
        model_instance = ModelInstanceIndex(i)
        model_instance_name = plant.GetModelInstanceName(model_instance)

        if model_instance_name.startswith(iiwa_prefix):
            num_iiwa_positions = plant.num_positions(model_instance)

            # I need a PassThrough system so that I can export the input port.
            iiwa_position = builder.AddSystem(PassThrough(num_iiwa_positions))
            builder.ExportInput(iiwa_position.get_input_port(),
                                model_instance_name + "_position")
            builder.ExportOutput(iiwa_position.get_output_port(),
                                 model_instance_name + "_position_command")

            # Export the iiwa "state" outputs.
            demux = builder.AddSystem(
                Demultiplexer(2 * num_iiwa_positions, num_iiwa_positions))
            builder.Connect(plant.get_state_output_port(model_instance),
                            demux.get_input_port())
            builder.ExportOutput(demux.get_output_port(0),
                                 model_instance_name + "_position_measured")
            builder.ExportOutput(demux.get_output_port(1),
                                 model_instance_name + "_velocity_estimated")
            builder.ExportOutput(plant.get_state_output_port(model_instance),
                                 model_instance_name + "_state_estimated")

            # Make the plant for the iiwa controller to use.
            controller_plant = MultibodyPlant(time_step=time_step)
            # TODO: Add the correct IIWA model (introspected from MBP)
            if plant.num_positions(model_instance) == 3:
                controller_iiwa = AddPlanarIiwa(controller_plant)
            else:
                controller_iiwa = AddIiwa(controller_plant)
            AddWsg(controller_plant, controller_iiwa, welded=True)
            controller_plant.Finalize()

            # Add the iiwa controller
            iiwa_controller = builder.AddSystem(
                InverseDynamicsController(controller_plant,
                                          kp=[100] * num_iiwa_positions,
                                          ki=[1] * num_iiwa_positions,
                                          kd=[20] * num_iiwa_positions,
                                          has_reference_acceleration=False))
            iiwa_controller.set_name(model_instance_name + "_controller")
            builder.Connect(plant.get_state_output_port(model_instance),
                            iiwa_controller.get_input_port_estimated_state())

            # Add in the feed-forward torque
            adder = builder.AddSystem(Adder(2, num_iiwa_positions))
            builder.Connect(iiwa_controller.get_output_port_control(),
                            adder.get_input_port(0))

            # Use a PassThrough to make the port optional (it will provide zero
            # values if not connected).
            torque_passthrough = builder.AddSystem(
                PassThrough([0] * num_iiwa_positions))
            builder.Connect(torque_passthrough.get_output_port(),
                            adder.get_input_port(1))
            builder.ExportInput(torque_passthrough.get_input_port(),
                                model_instance_name + "_feedforward_torque")
            builder.Connect(adder.get_output_port(),
                            plant.get_actuation_input_port(model_instance))

            # Add discrete derivative to command velocities.
            desired_state_from_position = builder.AddSystem(
                StateInterpolatorWithDiscreteDerivative(
                    num_iiwa_positions,
                    time_step,
                    suppress_initial_transient=True))
            desired_state_from_position.set_name(
                model_instance_name + "_desired_state_from_position")
            builder.Connect(desired_state_from_position.get_output_port(),
                            iiwa_controller.get_input_port_desired_state())
            builder.Connect(iiwa_position.get_output_port(),
                            desired_state_from_position.get_input_port())

            # Export commanded torques.
            builder.ExportOutput(adder.get_output_port(),
                                 model_instance_name + "_torque_commanded")
            builder.ExportOutput(adder.get_output_port(),
                                 model_instance_name + "_torque_measured")

            builder.ExportOutput(
                plant.get_generalized_contact_forces_output_port(
                    model_instance), model_instance_name + "_torque_external")

        elif model_instance_name.startswith(wsg_prefix):

            # Wsg controller.
            wsg_controller = builder.AddSystem(SchunkWsgPositionController())
            wsg_controller.set_name(model_instance_name + "_controller")
            builder.Connect(wsg_controller.get_generalized_force_output_port(),
                            plant.get_actuation_input_port(model_instance))
            builder.Connect(plant.get_state_output_port(model_instance),
                            wsg_controller.get_state_input_port())
            builder.ExportInput(
                wsg_controller.get_desired_position_input_port(),
                model_instance_name + "_position")
            builder.ExportInput(wsg_controller.get_force_limit_input_port(),
                                model_instance_name + "_force_limit")
            wsg_mbp_state_to_wsg_state = builder.AddSystem(
                MakeMultibodyStateToWsgStateSystem())
            builder.Connect(plant.get_state_output_port(model_instance),
                            wsg_mbp_state_to_wsg_state.get_input_port())
            builder.ExportOutput(wsg_mbp_state_to_wsg_state.get_output_port(),
                                 model_instance_name + "_state_measured")
            builder.ExportOutput(wsg_controller.get_grip_force_output_port(),
                                 model_instance_name + "_force_measured")

class Setup():
    def __init__(self, time_step = 0.002):
        self.builder = DiagramBuilder()
        self.plant, self.scene_graph = AddMultibodyPlantSceneGraph(self.builder, time_step = time_step)

        AddIiwas(self.plant) # load in two Iiwa's from model directives
        chalk_instance = AddEnvironment(self.plant)
        self.plant.Finalize() #finalize plant

        '''chalk_controller_plant = ChalkController()
        self.chalk_controller_plant = chalk_controller_plant
        self.chalk_instance = chalk_instance
#
        controller = self.builder.AddSystem(
            InverseDynamicsController(self.chalk_controller_plant,
                                    kp=[20] * 3,
                                    ki=[0] * 3,
                                    kd=[10] * 3,
                                    has_reference_acceleration=False))
        #set_trace()

        

        self.builder.Connect(self.plant.get_state_output_port(self.chalk_instance),
                        controller.get_input_port_estimated_state())
        self.builder.Connect(controller.get_output_port(),
                        self.plant.get_actuation_input_port(self.chalk_instance))'''

        IiwaControllerSetup(self.builder, self.plant, self.scene_graph, time_step = time_step)

        self.builder.ExportOutput(self.scene_graph.get_query_output_port(), "geometry_query")
        self.builder.ExportOutput(self.plant.get_contact_results_output_port(), 
                            "contact_results")
        self.builder.ExportOutput(self.plant.get_state_output_port(), 
                            "plant_continuous_state")
    def build_diagram(self):
        diagram = self.builder.Build()
        self.diagram = diagram
        pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].write_svg("diagram.svg")
        return diagram


class PseudoInverseController(LeafSystem):
    def __init__(self, plant, number = "0"):
        print(number)
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa"+number)
        print(self._iiwa)
        self._G = plant.GetBodyByName("body", plant.GetModelInstanceByName("wsg"+number)).body_frame()
        self._W = plant.world_frame()

        self.DeclareVectorInputPort("iiwa"+number+"_position", 7)
        self.DeclareVectorOutputPort("iiwa"+number+"_velocity", 7,
                                     self.CalcOutput)

    def CalcOutput(self, context, output):
        q = self.get_input_port().Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kQDot,
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,0:7] # Ignore gripper terms

        V_G_desired = np.array([0,    # rotation about x
                                -.1,  # rotation about y
                                0,    # rotation about z
                                0.05,    # x
                                -.05, # y
                                -.1]) # z
        v = np.linalg.pinv(J_G).dot(V_G_desired)
        output.SetFromVector(v)

def MescatViz(meshcat):
    builder = DiagramBuilder()
    setup = Setup() ## TODO Create scenario for two robotic arms
    station = builder.AddSystem(setup.build_diagram())
    #traj_source = builder.AddSystem(PoseTrajectorySource(traj))
    #builder.Connect(traj_source.get_output_port(),
    #                        station.GetInputPort("wsg0_position"))
    plant = station.GetSubsystemByName("plant")

    controller = builder.AddSystem(PseudoInverseController(plant, "0"))
    integrator = builder.AddSystem(Integrator(7))

    builder.Connect(controller.get_output_port(),
                    integrator.get_input_port())
    builder.Connect(integrator.get_output_port(),
                    station.GetInputPort("iiwa0_position"))
    builder.Connect(station.GetOutputPort("iiwa0_position_measured"),
                    controller.get_input_port())

    controller2 = builder.AddSystem(PseudoInverseController(plant, "1"))
    integrator2 = builder.AddSystem(Integrator(7))

    builder.Connect(controller2.get_output_port(),
                    integrator2.get_input_port())
    builder.Connect(integrator2.get_output_port(),
                    station.GetInputPort("iiwa1_position"))
    builder.Connect(station.GetOutputPort("iiwa1_position_measured"),
                    controller2.get_input_port())

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("geometry_query"), meshcat)

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    station_context = station.GetMyMutableContextFromRoot(context)

    plant = station.GetSubsystemByName("plant")
    controller_plant = station.GetSubsystemByName(
        "iiwa0_controller").get_multibody_plant_for_control()
    plant_context = plant.GetMyMutableContextFromRoot(context)

    q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName('iiwa0'))
    #station.GetInputPort('iiwa0_position').FixValue(station_context, q0)
    #station.GetInputPort('iiwa1_position').FixValue(station_context, q0)

    #station.GetInputPort('wsg0_position').FixValue(station_context, [0])
    #station.GetInputPort('wsg1_position').FixValue(station_context, [0])

    station.GetInputPort("iiwa0_feedforward_torque").FixValue(
        station_context, np.zeros((7, 1)))
    station.GetInputPort("wsg0_position").FixValue(station_context,[0])
    integrator.set_integral_value(
        integrator.GetMyContextFromRoot(context),
        plant.GetPositions(plant.GetMyContextFromRoot(context),
                           plant.GetModelInstanceByName("iiwa0")))

    station.GetInputPort("iiwa1_feedforward_torque").FixValue(
        station_context, np.zeros((7, 1)))
    station.GetInputPort("wsg1_position").FixValue(station_context,[0])
    integrator2.set_integral_value(
        integrator2.GetMyContextFromRoot(context),
        plant.GetPositions(plant.GetMyContextFromRoot(context),
                           plant.GetModelInstanceByName("iiwa1")))

    visualizer.StartRecording(False)
    simulator.AdvanceTo(20.0 if running_as_notebook else 0.1);
    visualizer.PublishRecording()

    return builder, station

In [ ]:
# Circle Trajectory Moving
class PoseTrajectorySource(LeafSystem):
    def __init__(self, pose_trajectory):
        LeafSystem.__init__(self)
        self._pose_trajectory = pose_trajectory
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()),
            self.CalcPose)

    def CalcPose(self, context, output):
        output.set_value(self._pose_trajectory.GetPose(context.get_time()))
        
radius = 0.2

p0 = [-0.45, 0.0, 0.4]
p1 = [-0.45, 0.0, 1.0]
R0 = RotationMatrix(np.array([[0, 1, 0], [0, 0, -1], [-1, 0, 0]]).T)
X_WorldCenter = RigidTransform(R0, p0)
X_WorldGripper_init = RigidTransform(R0, p1)

num_key_frames = 10
total_time = 20

thetas = np.linspace(0, 2*np.pi, num_key_frames)

def visualize_key_frames(frame_poses):
    for i, pose in enumerate(frame_poses):
        AddMeshcatTriad(meshcat, 'frame_{}'.format(i),
                        length=0.15, radius=0.006, X_PT=pose)

def compose_circular_key_frames(thetas, X_WorldCenter, X_WorldGripper_init):
    key_frame_poses_in_world = [X_WorldGripper_init]
    for theta in thetas:
        p = [0.1*-1*np.sin(theta), 0.0 , 0.1*-1*np.cos(theta)]
        R = RotationMatrix.MakeYRotation(theta)
        center_pose = RigidTransform(R,p)
        this_pose = X_WorldCenter.multiply(center_pose)
        key_frame_poses_in_world.append(this_pose)
        
    return key_frame_poses_in_world

key_frame_poses = compose_circular_key_frames(thetas, X_WorldCenter, X_WorldGripper_init)  

visualize_key_frames(key_frame_poses)

times = np.linspace(0, total_time, num_key_frames+1)
traj = PiecewisePose.MakeLinear(times, key_frame_poses)


In [7]:
builder, station = MescatViz(meshcat)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ba8e65c7-ba08-42d3-b249-70b2fa081e19' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>